## CCF: 企业非法集资风险预测

In [8]:
import time
import math
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb
from util import *

t_base_info = "./dataset/train/base_info.csv"
t_annual_report_info = "./dataset/train/annual_report_info.csv"
t_tax_info = "./dataset/train/tax_info.csv"
t_change_info = "./dataset/train/change_info.csv"
t_news_info = "./dataset/train/news_info.csv"
t_other_info = "./dataset/train/other_info.csv"
t_entprise_info = "./dataset/train/entprise_info.csv" # train labeled data

In [9]:
print("Reading...")
# base_info = pd.read_csv('dataset/train/base_info.csv')

Reading...


In [10]:
# base_info.describe()

### entprise_info

In [11]:
df_label_info = pd.read_csv(t_entprise_info, header=0)

df_label_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14865 entries, 0 to 14864
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      14865 non-null  object
 1   label   14865 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 232.4+ KB


In [12]:
df_label_info['label'].unique()
df_label_info['id'].unique()
df_label_info['label'].value_counts()
df_label_info['id'].value_counts()


59b38c56de3836830e01bbfc9d637352fca123d55c0c4840    1
d8071a739aa75a3b6e10545a01527e65ca25a2ab649bf230    1
f000950527a6feb619d2e5dc9f09a14b1caf8a3d54240e5b    1
59b38c56de383683777509481858dcd37519ff7776ccc5cf    1
f000950527a6feb681588a59f998db9966778dabfa1f7f4b    1
                                                   ..
755db3b5c5f74eb4512bf9e7830426b50c8bfb581174ea43    1
f000950527a6feb65d91c5dfb7459922c227f09010bc5ad0    1
f1c1045b13d18329d8822d516660007659ea3ad68f24cca4    1
82750f1b9d122350bab2e8b123142477b1a9bc2d7678c527    1
d8071a739aa75a3be9225cff8393ef95165f1e17abd62dfe    1
Name: id, Length: 14865, dtype: int64

### other_info.csv

In [26]:
# TODO: 特征空值过多 如何处理？ #
df_other_info = pd.read_csv(t_other_info, header=0)
df_other_info['legal_judgment_num'].value_counts()
df_other_info.shape
df_other_info.head(5)

,id,legal_judgment_num,brand_num,patent_num
0,f000950527a6feb6d340f91da09e61347d8200cd2f0d1602,4.0,NaN,NaN
1,f000950527a6feb608dd9322b74a99f60851207f36a3c94c,1.0,NaN,NaN
2,d8071a739aa75a3b9f23966f8dae78fd226c272515b9c255,2.0,NaN,NaN
3,216bd2aaf4d079242209b1496f81a36c7abed9dd0bb65ed3,NaN,1.0,NaN
4,e9f7b28ec10e0470de9631c789f49acdd4e7cf9ed6db094b,NaN,2.0,NaN


In [27]:
# 统计空值的个数
null_cnt_lj = df_other_info['legal_judgment_num'].isnull().sum(axis=0)
null_cnt_br = df_other_info['brand_num'].isnull().sum(axis=0)
null_cnt_pt = df_other_info['patent_num'].isnull().sum(axis=0)
print(null_cnt, null_cnt_br, null_cnt_pt)


884 981 1494


In [95]:
training_set = pd.merge(df_label_info, df_other_info, how='left', on='id')
# 释放内存
del df_label_info
del df_other_info
# training_set['id'].value_counts()
training_set = training_set.groupby(['id'], as_index=False).mean()
# training_set[training_set['id'] == "f000950527a6feb63702b1f6c1dabe5ea196d320bbbff425"]
training_set.shape

NameError: name 'df_label_info' is not defined

### news_info.csv

In [15]:
#  TODO: 随着时间变化，news的态度不同 但是标签只能是一个#
df_news_info = pd.read_csv(t_news_info, header=0)

dict_atitude = {"积极": 0, "中立": 1, "消极": 2}
df_news_info['positive_negtive'] = df_news_info['positive_negtive'].map(lambda x : dict_atitude[x])
# public date 转变成迄今为止发生时间
cmp_date = "2020-10-09"
from util import *

# df_news_info['public_date'] = df_news_info['public_date'].map(lambda x: check_date(x))
# df_news_info['public_date'].value_counts()
# 处理"xx小时前" 数据统一为昨天更新 
def handle_public_date(str):
    if(check_date(str) == False):
        return 1
    else:
        return days(cmp_date, str)
df_news_info['public_date'] = df_news_info['public_date'].map(lambda x: handle_public_date(x))
df_news_info.shape

time data '4小时前' does not match format '%Y-%m-%d'
time data '14小时前' does not match format '%Y-%m-%d'
time data '25分钟前' does not match format '%Y-%m-%d'
time data '30分钟前' does not match format '%Y-%m-%d'
time data '1小时前' does not match format '%Y-%m-%d'
time data '2小时前' does not match format '%Y-%m-%d'
time data '2小时前' does not match format '%Y-%m-%d'
time data '8小时前' does not match format '%Y-%m-%d'
time data '9小时前' does not match format '%Y-%m-%d'
time data '22小时前' does not match format '%Y-%m-%d'
time data '23小时前' does not match format '%Y-%m-%d'
time data '6小时前' does not match format '%Y-%m-%d'
time data '5小时前' does not match format '%Y-%m-%d'
time data '8小时前' does not match format '%Y-%m-%d'
time data '8小时前' does not match format '%Y-%m-%d'
time data '9小时前' does not match format '%Y-%m-%d'


(10518, 3)

In [18]:
training_set = pd.merge(training_set, df_news_info, how='left', on='id')
training_set['id'].value_counts()

f000950527a6feb63702b1f6c1dabe5ea196d320bbbff425    1193
d8071a739aa75a3b04c9a1789bd657ef83e10c02b1a9fef5     290
f000950527a6feb6194eb25023f715fa9ec0166d20cc8aed     215
f000950527a6feb6c0db441978864a6a0f04668a61d0bab6     202
f000950527a6feb6375809f26c0e7fa103f56259da2e53ac     160
                                                    ... 
59b38c56de383683bb52e42f81ee59ff54829b6a35cd7405       1
d8071a739aa75a3b6eea69adcb8ac0aa156b002cb8cec99a       1
216bd2aaf4d07924c692faa474afe4b4bf531635e503dcdc       1
f000950527a6feb684a4a1834bc09a886b678fc0ef24180b       1
d8071a739aa75a3be9225cff8393ef95165f1e17abd62dfe       1
Name: id, Length: 14865, dtype: int64

In [29]:
# 一个id对应不同时间的不同新闻评价
tmp = training_set[training_set['id'] == "f000950527a6feb63702b1f6c1dabe5ea196d320bbbff425"]
tmp.head(100)
training_set.shape
training_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19407 entries, 0 to 19406
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  19407 non-null  object 
 1   label               19407 non-null  int64  
 2   legal_judgment_num  3258 non-null   float64
 3   brand_num           2924 non-null   float64
 4   patent_num          2334 non-null   float64
 5   positive_negtive    4930 non-null   float64
 6   public_date         4930 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 1.8+ MB


### change_info.csv

In [41]:
# TODO: 变更信息如何处理  这里直接删除 只保留变更时间 #
df_change_info = pd.read_csv(t_change_info, header=0)
# df_change_info['bqyf'] = df_change_info[['bgq', 'bgh']].apply(lambda x: x['bgq'] == x['bgh'], axis=1)
# del df_change_info[]
# df_change_info.shape
del df_change_info['bgxmdm']
del df_change_info['bgq']
del df_change_info['bgh']
df_change_info['bgrq'] = df_change_info['bgrq'].apply(np.log)
# 平滑 bgrq:变更日期
df_change_info.head()

,id,bgrq
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,30.636238
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,30.636238
2,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,30.635224
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,30.635224
4,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,30.635224


### tax_info.csv


In [88]:
# TODO: 很多统计信息， 需要去提取特征 #
df_tax_info = pd.read_csv(t_tax_info, header=0)
df_tax_info['TAX_CATEGORIES'].value_counts()
df_tax_info['TAX_ITEMS'].value_counts()
df_tax_info['TAX_AMOUNT'].value_counts()
df_tax_info['TAXATION_BASIS'].value_counts()

df_tax_info['TAX_DAYS'] = df_tax_info[['START_DATE', 'END_DATE']].apply(lambda x : days_v1(x['END_DATE'], x['START_DATE']), axis=1)

del df_tax_info['START_DATE']
del df_tax_info['END_DATE']

tax_cg_dict = {}
cnt = 0
for e in df_tax_info['TAX_CATEGORIES']:
    if e in tax_cg_dict: continue
    else:
        tax_cg_dict[e] = cnt 
        cnt += 1

df_tax_info['TAX_CATEGORIES'] = df_tax_info['TAX_CATEGORIES'].map(lambda x : tax_cg_dict[x])

tax_it_dict = {}
cnt = 0
for e in df_tax_info['TAX_ITEMS']:
    if e in tax_it_dict: continue
    else:
        tax_it_dict[e] = cnt 
        cnt += 1
df_tax_info['TAX_ITEMS'] = df_tax_info['TAX_ITEMS'].map(lambda x : tax_it_dict[x])
df_tax_info['TAXATION_BASIS'] = df_tax_info['TAXATION_BASIS'].apply(np.log)

df_tax_info.head()


,id,TAX_CATEGORIES,TAX_ITEMS,TAXATION_BASIS,TAX_RATE,DEDUCTION,TAX_AMOUNT,TAX_DAYS
0,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,0,0,11.191766,0.0003,-0.04,21.8,29
1,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,0,1,11.191766,0.0003,-0.04,21.8,29
2,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,0,2,11.191766,0.0003,-0.04,21.8,29
3,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,0,3,11.191766,0.0003,-0.04,21.8,29
4,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,0,4,11.191766,0.0003,-0.04,21.8,29


In [92]:
training_set = pd.merge(training_set, df_tax_info, how='left', on='id')
# training_set['id'].value_counts()
# training_set.info()
training_set.shape

MemoryError: Unable to allocate 750. MiB for an array with shape (98356341,) and data type int64

### annual_report_info.csv

In [100]:
df_anreport_info = pd.read_csv(t_annual_report_info, header=0)
df_anreport_info['ANCHEYEAR'].value_counts()
df_anreport_info['STATE'].value_counts()
df_anreport_info.head()



,id,ANCHEYEAR,STATE,FUNDAM,MEMNUM,FARNUM,ANNNEWMEMNUM,ANNREDMEMNUM,EMPNUM,EMPNUMSIGN,...,DISPERNUM,UNENUM,COLEMPLNUM,RETEMPLNUM,DISEMPLNUM,UNEEMPLNUM,WEBSITSIGN,FORINVESTSIGN,STOCKTRANSIGN,PUBSTATE
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2017.0,2.0,5.0,NaN,NaN,NaN,NaN,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2018.0,2.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
2,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2017.0,2.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
3,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2018.0,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,...,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0
4,9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17,2017.0,2.0,5.0,NaN,NaN,NaN,NaN,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,3.0
